In [1]:
pip install meteostat

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install fuzzywuzzy[speedup] pandas

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import requests
import json
from datetime import datetime
import matplotlib.pyplot as plt
from meteostat import Point, Daily
import pandas as pd
import matplotlib.pyplot as plt
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import time
import folium
from folium import Marker
from folium.plugins import HeatMap
from fuzzywuzzy import process

In [4]:
# Read CSV file
uni_data = pd.read_csv('merged_datasets.csv')

uni_data.head(50)

,University Name,In State Tuition,Out of State Tuition,Admission Rate,Number of Students,Graduation Rate 4-years,Earnings After 10 Years,Retention Rate,Diversity Score,Public or Private,...,er score,isr score,fsr score,ifr score,Acres,Safety,Facilities,Social,Average Sentiment,University ID
0,Princeton University,57410,57410,0.0570,5527,0.9744,110066,0.9705,0.701479,0,...,97.5,57.5,64.5,16.7,600.0,4.5,4.4,4.1,0.250584,1
1,Massachusetts Institute of Technology,57986,57986,0.0396,4638,0.9557,143372,0.9898,0.699191,0,...,100.0,88.2,100.0,100.0,168.0,4.5,3.7,3.7,0.468943,2
2,Harvard University,57261,57261,0.0324,7973,0.9726,101817,0.9769,0.703174,0,...,100.0,66.8,98.3,84.6,209.0,3.2,3.2,2.9,0.035685,3
3,Stanford University,58416,58416,0.0368,7761,0.9534,124080,0.9831,0.694972,0,...,100.0,51.2,100.0,99.9,8180.0,4.5,4.5,4.0,0.595871,4
4,Yale University,62250,62250,0.0457,6639,0.9736,100533,0.9810,0.702715,0,...,99.9,78.8,100.0,89.8,1108.0,3.6,4.1,4.0,0.427826,5
5,California Institute of Technology,19190,19190,0.0270,540,0.7828,53598,0.9474,0.483280,0,...,93.7,81.0,100.0,100.0,124.0,4.2,4.3,3.6,0.137369,6
6,Duke University,62688,62688,0.0635,6570,0.9625,97800,0.9685,0.700444,0,...,69.3,52.2,100.0,16.8,8693.0,4.4,4.8,4.4,0.796632,7
7,Johns Hopkins University,60480,60480,0.0725,5643,0.9433,87555,0.9695,0.688898,0,...,59.7,91.4,100.0,64.4,140.0,3.2,4.3,3.3,0.417095,8
8,Northwestern University,8672,19460,0.9793,6243,0.4528,47021,0.6115,0.600800,1,...,60.2,44.2,98.9,48.2,242.8,4.3,4.3,3.7,0.267606,9
9,University of Pennsylvania,63452,63452,0.0650,10572,0.9615,111371,0.9837,0.709739,0,...,89.1,70.3,99.9,96.0,299.0,3.5,4.2,4.2,0.334003,10


In [5]:
standard_names = ['Princeton University',
 'Massachusetts Institute of Technology',
 'Harvard University',
 'Stanford University',
 'Yale University',
 'California Institute of Technology',
 'Duke University',
 'Johns Hopkins University',
 'Northwestern University',
 'University of Pennsylvania',
 'Cornell University',
 'University of Chicago',
 'Brown University',
 'Columbia University',
 'Dartmouth College',
 'University of California, Los Angeles',
 'University of California, Berkeley',
 'Rice University',
 'University of Notre Dame',
 'Vanderbilt University',
 'Carnegie Mellon University',
 'University of Michigan, Ann Arbor',
 'Washington University in St. Louis',
 'Emory University',
 'Georgetown University',
 'University of Virginia',
 'University of North Carolina, Chapel Hill',
 'University of Southern California',
 'University of California, San Diego',
 'New York University',
 'University of Florida',
 'University of Texas, Austin',
 'Georgia Institute of Technology',
 'University of California, Davis',
 'University of California, Irvine',
 'University of Illinois, Urbana-Champaign',
 'Boston College',
 'Tufts University',
 'University of California, Santa Barbara',
 'University of Wisconsin, Madison',
 'Boston University',
 'Ohio State University, Columbus',
 'Rutgers University, New Brunswick',
 'University of Maryland, College Park',
 'University of Rochester',
 'Lehigh University',
 'Purdue University',
 'University of Georgia',
 'University of Washington',
 'Wake Forest University']

In [6]:
def standardize_column_values(df, column, standard_names, threshold=80):

    updated_values = []
    
    for value in df[column]:
        match, score = process.extractOne(value, standard_names)
        if score >= threshold:
            updated_values.append(match)  # Use the matched standard name
        else:
            updated_values.append(value)  # Keep the original name if no good match
    
    df[column] = updated_values
    return df

In [7]:
uni_data = standardize_column_values(uni_data, 'University Name', standard_names, threshold=80)

In [8]:
uni_data.head(50)

,University Name,In State Tuition,Out of State Tuition,Admission Rate,Number of Students,Graduation Rate 4-years,Earnings After 10 Years,Retention Rate,Diversity Score,Public or Private,...,er score,isr score,fsr score,ifr score,Acres,Safety,Facilities,Social,Average Sentiment,University ID
0,Princeton University,57410,57410,0.0570,5527,0.9744,110066,0.9705,0.701479,0,...,97.5,57.5,64.5,16.7,600.0,4.5,4.4,4.1,0.250584,1
1,Massachusetts Institute of Technology,57986,57986,0.0396,4638,0.9557,143372,0.9898,0.699191,0,...,100.0,88.2,100.0,100.0,168.0,4.5,3.7,3.7,0.468943,2
2,Harvard University,57261,57261,0.0324,7973,0.9726,101817,0.9769,0.703174,0,...,100.0,66.8,98.3,84.6,209.0,3.2,3.2,2.9,0.035685,3
3,Stanford University,58416,58416,0.0368,7761,0.9534,124080,0.9831,0.694972,0,...,100.0,51.2,100.0,99.9,8180.0,4.5,4.5,4.0,0.595871,4
4,Yale University,62250,62250,0.0457,6639,0.9736,100533,0.9810,0.702715,0,...,99.9,78.8,100.0,89.8,1108.0,3.6,4.1,4.0,0.427826,5
5,California Institute of Technology,19190,19190,0.0270,540,0.7828,53598,0.9474,0.483280,0,...,93.7,81.0,100.0,100.0,124.0,4.2,4.3,3.6,0.137369,6
6,Duke University,62688,62688,0.0635,6570,0.9625,97800,0.9685,0.700444,0,...,69.3,52.2,100.0,16.8,8693.0,4.4,4.8,4.4,0.796632,7
7,Johns Hopkins University,60480,60480,0.0725,5643,0.9433,87555,0.9695,0.688898,0,...,59.7,91.4,100.0,64.4,140.0,3.2,4.3,3.3,0.417095,8
8,Northwestern University,8672,19460,0.9793,6243,0.4528,47021,0.6115,0.600800,1,...,60.2,44.2,98.9,48.2,242.8,4.3,4.3,3.7,0.267606,9
9,University of Pennsylvania,63452,63452,0.0650,10572,0.9615,111371,0.9837,0.709739,0,...,89.1,70.3,99.9,96.0,299.0,3.5,4.2,4.2,0.334003,10


In [9]:
def find_coordinates(location_name):
    """
    Finds the latitude and longitude of a given location name.
    
    Args:
        location_name (str): The name of the location.
    
    Returns:
        tuple: A tuple containing latitude and longitude.
               Returns None if the location is not found.
    """
    geolocator = Nominatim(user_agent="geo_locator")
    try:
        location = geolocator.geocode(location_name)
        return location.latitude, location.longitude
    except GeocoderTimedOut:
        time.sleep(1)  # Retry after a short delay
        return find_coordinates(location_name)

In [10]:
# Apply the function to find coordinates and add new columns
uni_data[['Latitude', 'Longitude']] = uni_data['University Name'].apply(lambda name: pd.Series(find_coordinates(name)))

In [11]:
uni_data.head(50)

,University Name,In State Tuition,Out of State Tuition,Admission Rate,Number of Students,Graduation Rate 4-years,Earnings After 10 Years,Retention Rate,Diversity Score,Public or Private,...,fsr score,ifr score,Acres,Safety,Facilities,Social,Average Sentiment,University ID,Latitude,Longitude
0,Princeton University,57410,57410,0.0570,5527,0.9744,110066,0.9705,0.701479,0,...,64.5,16.7,600.0,4.5,4.4,4.1,0.250584,1,40.338675,-74.658365
1,Massachusetts Institute of Technology,57986,57986,0.0396,4638,0.9557,143372,0.9898,0.699191,0,...,100.0,100.0,168.0,4.5,3.7,3.7,0.468943,2,42.358253,-71.096627
2,Harvard University,57261,57261,0.0324,7973,0.9726,101817,0.9769,0.703174,0,...,98.3,84.6,209.0,3.2,3.2,2.9,0.035685,3,42.365743,-71.122214
3,Stanford University,58416,58416,0.0368,7761,0.9534,124080,0.9831,0.694972,0,...,100.0,99.9,8180.0,4.5,4.5,4.0,0.595871,4,37.431314,-122.169365
4,Yale University,62250,62250,0.0457,6639,0.9736,100533,0.9810,0.702715,0,...,100.0,89.8,1108.0,3.6,4.1,4.0,0.427826,5,41.257131,-72.989670
5,California Institute of Technology,19190,19190,0.0270,540,0.7828,53598,0.9474,0.483280,0,...,100.0,100.0,124.0,4.2,4.3,3.6,0.137369,6,34.137014,-118.125288
6,Duke University,62688,62688,0.0635,6570,0.9625,97800,0.9685,0.700444,0,...,100.0,16.8,8693.0,4.4,4.8,4.4,0.796632,7,36.000156,-78.944230
7,Johns Hopkins University,60480,60480,0.0725,5643,0.9433,87555,0.9695,0.688898,0,...,100.0,64.4,140.0,3.2,4.3,3.3,0.417095,8,39.330202,-76.621854
8,Northwestern University,8672,19460,0.9793,6243,0.4528,47021,0.6115,0.600800,1,...,98.9,48.2,242.8,4.3,4.3,3.7,0.267606,9,42.055716,-87.675295
9,University of Pennsylvania,63452,63452,0.0650,10572,0.9615,111371,0.9837,0.709739,0,...,99.9,96.0,299.0,3.5,4.2,4.2,0.334003,10,39.949340,-75.189649


In [12]:
# Create a map centered on North America
map_na = folium.Map(location=[39.8283, -98.5795], tiles='cartodbpositron', zoom_start=4)

# Add university points to the map
for idx, row in uni_data.iterrows():
    folium.Marker(location=[row["Latitude"], row["Longitude"]],popup=f"{row['University Name']}").add_to(map_na)

# Display the map
map_na

In [13]:
# Define a function to create a heatmap
def create_heatmap(dataframe, column):
    # Initialize a map centered on North America
    heatmap = folium.Map(location=[39.8283, -98.5795], tiles='cartodbpositron', zoom_start=4)
    
    # Define the viridis color scale
    viridis_gradient = {
        0.0: '#440154',  # Dark purple
        0.2: '#3b528b',  # Blue
        0.4: '#21908d',  # Teal
        0.6: '#5ec962',  # Green
        0.8: '#fde725',  # Yellow
        1.0: '#ffffbf'   # Light yellow
    }
    
    # Extract the data for the heatmap (latitude, longitude, weight)
    heatmap_data = list(zip(dataframe["Latitude"], dataframe["Longitude"], dataframe[column]))
    
    # Add the HeatMap layer
    HeatMap(data=heatmap_data, radius=20, gradient=viridis_gradient).add_to(heatmap)
    
    return heatmap

In [14]:
# Create heatmaps for in-state and out-of-state tuition
heatmap_academic_reputation = create_heatmap(uni_data, "ar score")
heatmap_employee_reputation = create_heatmap(uni_data, "er score")

In [15]:
# Display heatmaps for academic reputation
heatmap_academic_reputation

In [16]:
# Display heatmaps for employee reputation
heatmap_employee_reputation